In [11]:
from google_drive_downloader import GoogleDriveDownloader as gdd

In [12]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1NkFzQ5N-I1J34-0uStvw0xaPbntqk7AV',
dest_path='content/rosseman.zip',   
unzip=True)

In [ ]:
# since the size of teras package is too big I have done Rnn in google collab.

In [2]:
from IPython.display import Markdown, display, Image
import warnings
warnings.filterwarnings('ignore')

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import numpy as np
from statsmodels.tsa.stattools import adfuller, acf, pacf
import pandas as pd
from sklearn.preprocessing import Normalizer, MinMaxScaler, StandardScaler, RobustScaler
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import datetime
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
sns.set()

In [7]:
def read_csv(csv_path, missing_values=[]):
  """
  reads csv file
  """
  try:
    df = pd.read_csv(csv_path, na_values=missing_values)
    print("file read as csv")
    return df
  except FileNotFoundError:
    print("file not found")

In [16]:
df_train_store = read_csv("content/rossman data/train_store.csv")
test = read_csv("content/rossman data/test.csv")

file read as csv
file read as csv


In [21]:
# A quick view a data on hand
display(df_train_store.head())


,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Month,Year,SalesperCustomer,Day,Weekends,Weekdays,Monthcategory
0,1,1,5263,555,1,1,0,1,2,0,1270.0,9.0,2008.0,0,0.0,0.0,0,7,2015,9.482883,31,0,1,1
1,2,1,6064,625,1,1,0,1,0,0,570.0,11.0,2007.0,1,13.0,2010.0,2,7,2015,9.702400,31,0,1,1
2,3,1,8314,821,1,1,0,1,0,0,14130.0,12.0,2006.0,1,14.0,2011.0,2,7,2015,10.126675,31,0,1,1
3,4,1,13995,1498,1,1,0,1,2,2,620.0,9.0,2009.0,0,0.0,0.0,0,7,2015,9.342457,31,0,1,1
4,5,1,4822,559,1,1,0,1,0,0,29910.0,4.0,2015.0,0,0.0,0.0,0,7,2015,8.626118,31,0,1,1


In [22]:
display(df_train_store.describe())


,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Month,Year,SalesperCustomer,Day,Weekends,Weekdays,Monthcategory
count,844338.000000,844338.0,844338.000000,844338.000000,844338.0,844338.000000,844338.000000,844338.000000,844338.000000,844338.000000,844338.000000,844338.000000,844338.000000,844338.000000,844338.000000,844338.000000,844338.000000,844338.000000,844338.000000,844338.000000,844338.000000,844338.0,844338.0,844338.000000
mean,558.421374,1.0,6955.959134,762.777166,1.0,0.446356,0.001418,0.193578,1.205441,0.936493,5450.044852,4.926482,1369.692738,0.498670,11.596159,1003.201259,0.976795,5.845774,2013.831945,9.493641,15.835706,0.0,1.0,1.016797
std,321.730861,0.0,3103.815515,401.194153,0.0,0.497114,0.047578,0.395102,1.363375,0.993099,7801.082007,4.283634,935.556484,0.499999,15.308101,1005.874685,1.081646,3.323959,0.777271,2.197448,8.683392,0.0,0.0,0.810570
min,1.000000,1.0,46.000000,8.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2013.000000,2.749075,1.000000,0.0,1.0,0.000000
25%,280.000000,1.0,4859.000000,519.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,710.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,2013.000000,7.895571,8.000000,0.0,1.0,0.000000
50%,558.000000,1.0,6369.000000,676.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,2325.000000,4.000000,2006.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2014.000000,9.250000,16.000000,0.0,1.0,1.000000
75%,837.000000,1.0,8360.000000,893.000000,1.0,1.000000,0.000000,0.000000,3.000000,2.000000,6880.000000,9.000000,2011.000000,1.000000,22.000000,2012.000000,2.000000,8.000000,2014.000000,10.899729,23.000000,0.0,1.0,2.000000
max,1115.000000,1.0,41551.000000,7388.000000,1.0,1.000000,3.000000,1.000000,3.000000,2.000000,75860.000000,12.000000,2015.000000,1.000000,50.000000,2015.000000,3.000000,12.000000,2015.000000,64.957854,31.000000,0.0,1.0,2.000000


In [23]:
display(df_train_store.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 844338 entries, 0 to 844337
Data columns (total 24 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Store                      844338 non-null  int64  
 1   DayOfWeek                  844338 non-null  int64  
 2   Sales                      844338 non-null  int64  
 3   Customers                  844338 non-null  int64  
 4   Open                       844338 non-null  int64  
 5   Promo                      844338 non-null  int64  
 6   StateHoliday               844338 non-null  int64  
 7   SchoolHoliday              844338 non-null  int64  
 8   StoreType                  844338 non-null  int64  
 9   Assortment                 844338 non-null  int64  
 10  CompetitionDistance        844338 non-null  float64
 11  CompetitionOpenSinceMonth  844338 non-null  float64
 12  CompetitionOpenSinceYear   844338 non-null  float64
 13  Promo2                     84

None